In [1]:
import json
import re

# 读取JSON文件
def read_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# 解析output_blocks中的周数
def parse_weeks(output_blocks):
    week_to_block = {}
    for block in output_blocks:
        match = re.search(r"Week (\d+)", block)
        if match:
            week_number = int(match.group(1))
            week_to_block[week_number] = block
    return week_to_block

# 生成检查内容的prompt
def create_prompts(week_checks, week_to_block):
    prompts = []
    for week_num, event_desc in week_checks.items():
        week_num = int(week_num)
        if week_num in week_to_block:
            prompts.append(week_to_block[week_num] + f"Does this description include the {event_desc}? Please answer with 'yes' or 'no' only.")
    return prompts


# 主函数
file_path = "/home/yuhao/THREADING-THE-NEEDLE/Evalution/results.json"
datas = read_json(file_path)
prompts_once= []
prompts_range = []
prompts_periodic = []
for data in datas:
    week_to_block = parse_weeks(data['output_blocks'])
    # 生成once, range, periodic的prompts
    
    prompts_once.extend(create_prompts(data['wweek_checks_once'], week_to_block))
    prompts_range.extend(create_prompts(data['week_checks_range'], week_to_block))
    prompts_periodic.extend(create_prompts(data['week_checks_periodic'], week_to_block))




In [3]:
prompts_once

[" Week 24 (June 11th - June 17th)\nJust celebrated my birthday surrounded by those I love. I enjoyed engaging freely in heartfelt conversations, adored laughter, and ended this incredibly memorable weekend by dedicating a few inspired hours to catch magical images on the elegant morning sea.\n\nDoes this description include the husband (yourself) birthday? Please answer with 'yes' or 'no' only.",
 " Week 7 (February 12th - February 18th)\nThis week, I'm taking it easy, focusing on family commitments and photography projects. My birthday's here, and I got an amazing surprise – a thoughtful gift from my family! Didn't let myself get caught up in getting too much; we should be grateful for the simplest joys in life. \nWish to capture life's true beauty through my lens. In awe of how our children grow, seeing them smile each day.\n\nDoes this description include the wife birthday? Please answer with 'yes' or 'no' only.",
 " Week 28 (July 9th - July 15th)\nSurrounded by thriving scenarios 

In [1]:


import time
from vllm import LLM, SamplingParams
import torch
import json

# Define the sampling parameters
sampling_params = SamplingParams(temperature=0.95, top_p=0.95, max_tokens=50, seed=42)

# Example lists of prompts


# Record the start time
start_time = time.time()

# Initialize the LLM with the specified model and configuration
llm = LLM(model="meta-llama/Meta-Llama-3.1-8B-Instruct", tensor_parallel_size=2)

def evaluate_accuracy(prompts, llm, sampling_params):
    # Generate responses using the LLM
    outputs = llm.generate(prompts, sampling_params)
    total = len(outputs)
    correct = sum([1 for output in outputs if 'yes' in output.outputs[0].text])
    return correct / total if total > 0 else 0

# Evaluate the accuracy for each set of prompts
acc_once = evaluate_accuracy(prompts_once, llm, sampling_params)
acc_range = evaluate_accuracy(prompts_range, llm, sampling_params)
acc_periodic = evaluate_accuracy(prompts_periodic, llm, sampling_params)

# Print the accuracy results
print("Accuracy for once:", acc_once)
print("Accuracy for range:", acc_range)
print("Accuracy for periodic:", acc_periodic)

# Print the elapsed time
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")


/home/yuhao/anaconda3/envs/easycontext/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/yuhao/anaconda3/envs/easycontext/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `PYTORCH_TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-08-02 20:08:03,769	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 08-02 20:08:04 config.py:715] Defaulting to use mp for distributed inference
WARNING 08-02 20:08:04 arg_utils.py:762] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 08-02 20:08:04 config.py:806] Chunked prefill is enabled with max_num_batched_tokens=512.
INFO 08-02 20:08:04 llm_engine.py:176] Initializing an LLM engine (v0.5.3.post1) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=131072, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  2.11it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.85it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  2.66it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.63it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:01<00:00,  2.45it/s]



INFO 08-02 20:08:09 model_runner.py:692] Loading model weights took 7.5122 GB
(VllmWorkerProcess pid=3944603) INFO 08-02 20:08:09 model_runner.py:692] Loading model weights took 7.5122 GB
INFO 08-02 20:08:10 distributed_gpu_executor.py:56] # GPU blocks: 62561, # CPU blocks: 4096
(VllmWorkerProcess pid=3944603) ERROR 08-02 20:08:10 multiproc_worker_utils.py:226] Exception in worker VllmWorkerProcess while processing method initialize_cache: CUDA out of memory. Tried to allocate 1.91 GiB. GPU  has a total capacity of 79.15 GiB of which 1.70 GiB is free. Process 3944009 has 8.71 GiB memory in use. Including non-PyTorch memory, this process has 68.71 GiB memory in use. Of the allocated memory 66.76 GiB is allocated by PyTorch, and 25.36 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cu

 Week 28 (July 9th - July 15th)
Surrounded by thriving scenarios amidst identical approaches of life lies beneath spring, relaxes midst dedicated fountain. With every moment spend in our city celebrated person whose expertise knows absolutely – nostalgia I needed thinking foundations, thoughts basically, navigating. Oh what why dramatically soft bang self-relevance caught sure prosperous instal clearer morning confidence dances upon watched discovery increasing attain pebbled rock reserve – discovered time while awaits organ palms compassion all into start at urge tongue grateful guardians bad planting.

Does this description include the child_1 birthday? Please answer with 'yes' or 'no' only.
 Week 28 (July 9th - July 15th)
Surrounded by thriving scenarios amidst identical approaches of life lies beneath spring, relaxes midst dedicated fountain. With every moment spend in our city celebrated person whose expertise knows absolutely – nostalgia I needed thinking foundations, thoughts ba